# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 21 2022 8:30AM,240001,10,Enova-10168,Emerginnova,Released
1,Jun 20 2022 3:39PM,240002,19,ADV80003445,"AdvaGen Pharma, Ltd",Released
2,Jun 20 2022 3:39PM,240002,19,ADV80003450,"AdvaGen Pharma, Ltd",Released
3,Jun 20 2022 3:39PM,240002,19,ADV80003451,"AdvaGen Pharma, Ltd",Released
4,Jun 20 2022 3:39PM,240002,19,ADV80003452,"AdvaGen Pharma, Ltd",Released
5,Jun 20 2022 3:39PM,240002,19,ADV80003453,"AdvaGen Pharma, Ltd",Released
6,Jun 20 2022 3:39PM,240002,19,ADV80003454,"AdvaGen Pharma, Ltd",Released
7,Jun 20 2022 3:39PM,240002,19,ADV80003456,"AdvaGen Pharma, Ltd",Released
8,Jun 20 2022 3:39PM,240002,19,ADV80003457,"AdvaGen Pharma, Ltd",Released
9,Jun 20 2022 3:39PM,240002,19,ADV80003458,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
51,239996,Released,1
52,239999,Released,1
53,240000,Released,1
54,240001,Released,1
55,240002,Released,14


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239996,NaN,NaN,1.0
239999,NaN,NaN,1.0
240000,NaN,NaN,1.0
240001,NaN,NaN,1.0
240002,NaN,NaN,14.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239892,0.0,0.0,1.0
239896,0.0,0.0,1.0
239897,0.0,0.0,1.0
239899,1.0,5.0,9.0
239900,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239892,0,0,1
239896,0,0,1
239897,0,0,1
239899,1,5,9
239900,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239892,0,0,1
1,239896,0,0,1
2,239897,0,0,1
3,239899,1,5,9
4,239900,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239892,,,1
1,239896,,,1
2,239897,,,1
3,239899,1,5,9
4,239900,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 21 2022 8:30AM,240001,10,Emerginnova
1,Jun 20 2022 3:39PM,240002,19,"AdvaGen Pharma, Ltd"
15,Jun 20 2022 3:33PM,240000,22,FUJIFILM Diosynth Biotechnologies Texas LLC
16,Jun 20 2022 3:28PM,239999,16,American International Chemical
17,Jun 20 2022 3:16PM,239995,10,Bio-PRF
61,Jun 20 2022 3:15PM,239996,10,Yusen – 3D Matrix
62,Jun 20 2022 3:15PM,239993,10,Eywa Pharma Inc.
66,Jun 20 2022 3:03PM,239992,21,"NBTY Global, Inc."
67,Jun 20 2022 2:43PM,239960,20,"ACI Healthcare USA, Inc."
92,Jun 20 2022 2:23PM,239989,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 21 2022 8:30AM,240001,10,Emerginnova,,,1
1,Jun 20 2022 3:39PM,240002,19,"AdvaGen Pharma, Ltd",,,14
2,Jun 20 2022 3:33PM,240000,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,,1
3,Jun 20 2022 3:28PM,239999,16,American International Chemical,,,1
4,Jun 20 2022 3:16PM,239995,10,Bio-PRF,,,44
5,Jun 20 2022 3:15PM,239996,10,Yusen – 3D Matrix,,,1
6,Jun 20 2022 3:15PM,239993,10,Eywa Pharma Inc.,,,4
7,Jun 20 2022 3:03PM,239992,21,"NBTY Global, Inc.",,,1
8,Jun 20 2022 2:43PM,239960,20,"ACI Healthcare USA, Inc.",,5,20
9,Jun 20 2022 2:23PM,239989,10,ISDIN Corporation,,,26


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 8:30AM,240001,10,Emerginnova,1,,
1,Jun 20 2022 3:39PM,240002,19,"AdvaGen Pharma, Ltd",14,,
2,Jun 20 2022 3:33PM,240000,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,,
3,Jun 20 2022 3:28PM,239999,16,American International Chemical,1,,
4,Jun 20 2022 3:16PM,239995,10,Bio-PRF,44,,
5,Jun 20 2022 3:15PM,239996,10,Yusen – 3D Matrix,1,,
6,Jun 20 2022 3:15PM,239993,10,Eywa Pharma Inc.,4,,
7,Jun 20 2022 3:03PM,239992,21,"NBTY Global, Inc.",1,,
8,Jun 20 2022 2:43PM,239960,20,"ACI Healthcare USA, Inc.",20,5,
9,Jun 20 2022 2:23PM,239989,10,ISDIN Corporation,26,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 8:30AM,240001,10,Emerginnova,1,,
1,Jun 20 2022 3:39PM,240002,19,"AdvaGen Pharma, Ltd",14,,
2,Jun 20 2022 3:33PM,240000,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,,
3,Jun 20 2022 3:28PM,239999,16,American International Chemical,1,,
4,Jun 20 2022 3:16PM,239995,10,Bio-PRF,44,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 8:30AM,240001,10,Emerginnova,1.0,NaN,NaN
1,Jun 20 2022 3:39PM,240002,19,"AdvaGen Pharma, Ltd",14.0,NaN,NaN
2,Jun 20 2022 3:33PM,240000,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,NaN,NaN
3,Jun 20 2022 3:28PM,239999,16,American International Chemical,1.0,NaN,NaN
4,Jun 20 2022 3:16PM,239995,10,Bio-PRF,44.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 8:30AM,240001,10,Emerginnova,1.0,0.0,0.0
1,Jun 20 2022 3:39PM,240002,19,"AdvaGen Pharma, Ltd",14.0,0.0,0.0
2,Jun 20 2022 3:33PM,240000,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,0.0,0.0
3,Jun 20 2022 3:28PM,239999,16,American International Chemical,1.0,0.0,0.0
4,Jun 20 2022 3:16PM,239995,10,Bio-PRF,44.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4319371,172.0,7.0,25.0
12,239979,1.0,0.0,0.0
15,479905,4.0,9.0,1.0
16,1439511,6.0,0.0,0.0
19,1439792,18.0,1.0,0.0
20,959768,45.0,45.0,1.0
21,719848,3.0,0.0,0.0
22,1199739,5.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4319371,172.0,7.0,25.0
1,12,239979,1.0,0.0,0.0
2,15,479905,4.0,9.0,1.0
3,16,1439511,6.0,0.0,0.0
4,19,1439792,18.0,1.0,0.0
5,20,959768,45.0,45.0,1.0
6,21,719848,3.0,0.0,0.0
7,22,1199739,5.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,172.0,7.0,25.0
1,12,1.0,0.0,0.0
2,15,4.0,9.0,1.0
3,16,6.0,0.0,0.0
4,19,18.0,1.0,0.0
5,20,45.0,45.0,1.0
6,21,3.0,0.0,0.0
7,22,5.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,172.0
1,12,Released,1.0
2,15,Released,4.0
3,16,Released,6.0
4,19,Released,18.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,25.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
Executing,7.0,0.0,9.0,0.0,1.0,45.0,0.0,0.0
Released,172.0,1.0,4.0,6.0,18.0,45.0,3.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,25.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,Executing,7.0,0.0,9.0,0.0,1.0,45.0,0.0,0.0
2,Released,172.0,1.0,4.0,6.0,18.0,45.0,3.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,25.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,Executing,7.0,0.0,9.0,0.0,1.0,45.0,0.0,0.0
2,Released,172.0,1.0,4.0,6.0,18.0,45.0,3.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()